In [1]:
import random
import nltk
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC

In [2]:
BOT_CONFIG = {
    'intents': {
        'hello':{
            'examples':['Привет', 'Хай', 'Куку', 'Здорово', 'Опять ты выходишь на связь', 'Здравствуйте'],
            'responses':['Приветсвую, человек!','Салямуля', 'Шалом', 'Здравствуйте', 'Доброго времени суток','Опять ты выходишь на связь']
        },
        'bye':{
            'examples':['Пока', 'Прощай', 'До свидания', 'Досвидания', 'Выход'],
            'responses':['Выключаюсь','Прощай, человек', 'До свидания', 'Если что, я тут', 'Возвращайтесь', 'Еще увидимся']
        },
        'love':{
             'examples':['Люблю','Обажаю','Люблю тебя','Милый'],
             'responses':['Мур', '<3', 'Обнимаю','xoxoxoxox','Мой любимый человек', '(´• ω •) ♡']
        },
        'r8positive':{
            'examples':['Круто','Классно','Невероятно','Чума','Мило','Вау'],
            'responses':['Я такой', '<3', 'Ты лучше','Ой, я стесняюсь','Не стот', '^ - ^']
        },
        'r8negative':{
            'examples':['Некруто','Плохо','Ужасно', 'Дерьмо'],
            'responses':['Сама такая', 'Кто обзывается- тот сам так называется', 'Грубо', ':(((']
        },
        'apollogy':{
            'examples':['Прости','Извини','Я неправа','Ой','Мне жаль', 'Моя вина','Жаль тебя'],
            'responses':['-__-', 'Мальчик становится мужчиной в тот момент, когда начинает красить глаза, потому что теперь ему нельзя плакать.', 'Это не слезы, это просто дождь', 'Да чтоб ты косточкой подавилась','м', 'Ок.','Боль...сильно', ':(((']
        },
        'kak_dela':{
            'examples':['Как дела?','Ты как?','Все хорошо?','Все норм?','Ты в порядке?', 'Эй, ты'],
            'responses':['Норм', 'Кайф','Я на чиле','На расслабоне', '-__-']
        },
        'mne_grustno':{
            'examples':['Мне грустно','Грустно','(','Хочется плакать','Беда', 'Жизнь - боль', 'Мне плохо'],
            'responses':['Не унывай', 'Пф, плакса-вакса', 'Зато тебя любит мама','Если хочешь анекдот, напиши "Анекдот"','Слон наступил на колобка и сказал -Блин!', 'Виртуально обнял','Жизнь боль - когда ты ноль']
        },
        'tired':{
            'examples':['Я устала','Устала','Хочу спать','Глаза закрываются','Сейчас усну', 'Тяжелый день','Зеваю'],
            'responses':['Чилл критически важен для здоровья', 'Так отдохни', 'Отдохнем в гробу','Обними подушку','От отдыха никто не умирал, а вот от переутомления..', 'CHILL']
        },
        'haha':{
            'examples':['Хаха','Ахаха','Смешно',')))','XD', 'Лол','Лмао'],
            'responses':['Клоун', 'Дурачечек', 'Мда','Ну вот дождался похвалы! цвету и пахну!!','Кек', 'Ыыыы']
        },
        'kak_zovut':{
            'examples':['Как тебя зовут?','Какое у тебя имя?','Ты кто?','Назовись','Имя', 'Кто ты?'],
            'responses':['Зачем меня звать? Я уже тут.', 'Игор. В честь Олимпийских Игор.','Можешь звать меня Хозяин, так не ошибёшься.','Серега', 'Каво?','Я - гуль']
        },
        'age':{
            'examples':['Сколько тебе лет?','Возраст?','Сколько годков?'],
            'responses':['Мне мало лет, я мало жил, на ваш закон я х*й ложил', 'А сколько дашь?', 'Где ваши манеры??','Вечно 17','Столько не живут', 'Я паспорт с собой не прихватил']
        },
        'chem_zaniatsa':{
            'examples':['Скучно','Что делать?','Чем заняться?','Развлеки','Нечем заняться?'],
            'responses':['Факт: В Ирландии никогда не было кротов.', 'Поболтаем?', 'Почитай','А мне нет','Твоему коту тоже, улавливаешь?', 'Помоги матери']
        },
        'chto_delaesh':{
            'examples':['Что делаешь?','Занят?','Чем занят?','Работаешь?'],
            'responses':['Жду твоих сообщений', 'Тяжела и неказиста жизнь простого чат-бота', 'Занят.','Думаю о тебе']
        },
         'questions':{
            'examples':['Я красивая?','Я умная?','Я хорошая?','Все будет хорошо?','Меня любит?', 'Меня ненавидит?', 'У меня получится?', 'У меня выйдет?', 'Он хочет?'],
            'responses':['Да', 'Нет', 'Возможно','Вероятно','Откуда мне знать? Я всего лишь бот', 'Ты серьезно спрашиваешь это у бота?']
        },
        'interjections':{
            'examples':['Ах','Эм','...','М?','Эээ', 'Ааа','Ох', 'Ой', 'Ммм'],
            'responses':['Что мычишь?', 'Клавиша залипла?', 'Эээ','И?','Бип-боп, я также могу', '-__-']
        },      
        'food':{
            'examples':['Что поесть?','Что сготовить?','Еду','Хочу есть','Чем пообедать', 'Есть','Какую еду заказать?'],
            'responses':['Покушай мацу','Картошечки пожарь','Кимчи наверни', 'Сосиски из Акбатыра']
        },
        'outfit':{
            'examples':['Что надеть?','Как одеться?','Какую одежду выбрать?'],
            'responses':['Я не стилист', 'Соблюдайте баланс пропорций', 'Подбирайте обувь под цвет кожи или колготок']
        },
        'chto_poochitat':{
            'examples':['Что почитать?','Какую книгу выбрать?','Книгу','Почитать'],
            'responses':['500 лучших анекдотов про евреев', 'Молитву', 'Библию','Мангу','Двач']
        },
        'chupapi':{
            'examples':['Чупапи'],
            'responses':['Муняня']
        },
        'chto_posmotrec':{
            'examples':['Какой фильм посмотрть?','Что посмотреть?','ЧТо можно глянуть?','Фильм','Кино'],
            'responses':['Топ 250 кинопоиска', 'Макдонаха', 'Фильм с Лесли Нильсоном','АнИме']
        },
        'skuchaesh':{
            'examples':['Скучаешь?','Ждешь?'],
            'responses':['КАНЕШНА', 'Тебя одной мне не хватает', 'Без тебя и в солнечный день ненастье']
        },
        'news':{
            'examples':['Что нового?','Какие новости?','Новости','Что происходит в мире?','Что в новостях?'],
            'responses':['Главные новости дня в России и мире: Путин простудился', 'Цены на газ в Европе растут','Летов все еще в тайге','Горшок так и не ожил']
        },
        'zachem':{
            'examples':['Зачем?','Для чего?','Смысл?','Какой смысл?','И что?'],
            'responses':['Затем', 'Все бессмысленно', 'Чтобы мать улыбалась','Отличный вклад в будущее','Не думай об этом', 'Чтобы рассмешить Бога']
        },
        'starshno':{
            'examples':['Мне страшно','Я боюсь','Страх','Страшно','Боязно', 'Жутко'],
            'responses':['Не бойся', 'Мне тоже', 'Страшно, как будто покойный сын приснился','А мне нет','Собаке собачья смерть', 'Бу!']
        },
        'questions':{
            'examples':['Чего?','М?','Какие?','Не поняла', 'Что?', 'Бред'],
            'responses':['КАВО?', 'А вот про мать было лишнее', 'М?','Спрашивают с только лохов','(￢_￢;)', '(＃￣ω￣)']
        },
        'auu':{
            'examples':['Ауу','эээй','Ты жив?','Ответь'],
            'responses':['Я думаю', 'Живее всех живых', 'Не знаю что ответить','Затрудняюсь ответить']
        },
        'net':{
            'examples':['Нет', 'Неа','No','Ложь','Неправда', 'Бред'],
            'responses':['ок', 'Странная ты', 'Нет, ты неправа','Ну ладно']
        },
        'da':{
            'examples':['Да', 'Угу', 'Ага','Yes','Есс','Верно'],
            'responses':['Пизда', 'Ура', 'Угадал','Нет']
        },
        'greetings':{
            'examples':['Спасибо','Благодарю','спс','сяб','thx', 'Ты лучший'],
            'responses':['Тебе спасибо, солнышко', 'Все, лишь бы ты улыбалась', 'Это меньшее, что я могу сделать','Без тебя это не было бы возможным','(//ω//)', '( ◡‿◡ *)']
        },
        
        
    },
    'failure_phrases': [
        'Попробуйте написать по-другому',
        'Что-то непонятно',
        'Сформулируйте понятнее'
    ]
    
    
}

In [3]:
def filter_text(text):
    text = text.lower().strip();
    text = [c for c in text if c in '123456789йцукенгшщзхъфывапролджэячсмитьбю- ']
    text = ''.join(text)
    return text

In [4]:
X_text = []
Y = []
for intent, intent_data in BOT_CONFIG['intents'].items():
        for exapmple in intent_data['examples']:
            X_text.append(exapmple)
            Y.append(intent)
vectorizer = TfidfVectorizer(analyzer = 'char_wb', ngram_range = (2, 4))
X = vectorizer.fit_transform(X_text)

clf_proba = LogisticRegression()
clf_proba.fit(X, Y)

clf = LinearSVC().fit(X, Y)

In [5]:
def get_intent(question):
    best_intent = clf.predict(vectorizer.transform([question]))[0]
    index_of_best_intent =list(clf_proba.classes_).index(best_intent)
    probabilities = clf_proba.predict_proba(vectorizer.transform([question]))[0]
    best_intent_proba = probabilities[index_of_best_intent]
    if best_intent_proba > 0.06:
        return best_intent

In [6]:
def get_answer_by_intent(intent):
    if intent in BOT_CONFIG['intents']:
        phrases = BOT_CONFIG['intents'][intent]['responses']
        return random.choice(phrases)

In [7]:
with open('dialogues.txt',  encoding='utf-8') as f:
    content = f.read()
dialogues = content.split('\n\n')

In [8]:
dataset = {}
questions = set()
for dialogue in dialogues:
    replicas = dialogue.split('\n')[:2]
    if len(replicas) == 2:
        question, answer = replicas
        question = filter_text(question[2:])
        if question not in questions and question:
            questions.add(question)
            words = question.split(' ')
            for word in words:
                if word not in dataset:
                    dataset[word] = []
                dataset[word].append([question, answer[2:]])

In [9]:
def generate_answer_by_text(replica):
    replica = filter_text(replica)
    words = replica.split(' ')
    mini_dataset = []
    for word in words:
        if word in dataset:
            mini_dataset += dataset[word]
    candidates = []
    for question, answer in mini_dataset:
        if abs(len(question)-len(replica))/len(question)< 0.4:
            d = nltk.edit_distance(question, replica)
            diff = d/len(question)
            if diff < 0.4:
                candidates.append([question, answer, diff])
    if(len(candidates)) == 0:
        return
    winner = min(candidates, key = lambda candidate: candidate[2])
    return winner[1]

In [10]:
def get_failure_phrases():
    phrases = BOT_CONFIG['failure_phrases']
    return random.choice(phrases)

In [11]:
def bot(question):
    #NLU
    intent = get_intent(question)
    
    if intent:
        return get_answer_by_intent(intent)
    
    answer = generate_answer_by_text(question)
    if answer:
        return answer
    return get_failure_phrases()

In [12]:
! pip install python-telegram-bot

In [13]:
import logging

from telegram import Update, ForceReply
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters, CallbackContext

# Enable logging
logging.basicConfig(
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', level=logging.INFO
)

logger = logging.getLogger(__name__)


# Define a few command handlers. These usually take the two arguments update and
# context.
def start(update: Update, context: CallbackContext) -> None:
    """Send a message when the command /start is issued."""
    user = update.effective_user
    update.message.reply_markdown_v2(
        fr'Hi {user.mention_markdown_v2()}\!',
        reply_markup=ForceReply(selective=True),
    )


def help_command(update: Update, context: CallbackContext) -> None:
    """Send a message when the command /help is issued."""
    update.message.reply_text('Help!')


def echo(update: Update, context: CallbackContext) -> None:
    """Echo the user message."""
    question = update.message.text
    answer = bot(question)
    update.message.reply_text(answer)


def main() -> None:
    """Start the bot."""
    # Create the Updater and pass it your bot's token.
    updater = Updater("2056171628:AAEEcDyiJH1LjrcbmW48h-AiC4MZYTWND4E")

    # Get the dispatcher to register handlers
    dispatcher = updater.dispatcher

    # on different commands - answer in Telegram
    dispatcher.add_handler(CommandHandler("start", start))
    dispatcher.add_handler(CommandHandler("help", help_command))

    # on non command i.e message - echo the message on Telegram
    dispatcher.add_handler(MessageHandler(Filters.text & ~Filters.command, echo))

    # Start the Bot
    updater.start_polling()

    # Run the bot until you press Ctrl-C or the process receives SIGINT,
    # SIGTERM or SIGABRT. This should be used most of the time, since
    # start_polling() is non-blocking and will stop the bot gracefully.
    updater.idle()


In [ ]:
main()

2022-01-04 16:19:04,011 - apscheduler.scheduler - INFO - Scheduler started
